In [1]:
! git clone https://github.com/singhnaveen098/Hamoye_capstone_project_smote.git
train_path = 'Hamoye_capstone_project_smote/Data/train/'
val_path = 'Hamoye_capstone_project_smote/Data/val/'
test_path = 'Hamoye_capstone_project_smote/Data/test/'

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from tensorflow.keras import Model

In [3]:
os.listdir('Hamoye_capstone_project_smote/Data/test/')

In [4]:
train_df = image_dataset_from_directory(train_path,label_mode='categorical',image_size=(256, 256))
test_df = image_dataset_from_directory(test_path,label_mode='categorical',image_size=(256, 256))
val_df = image_dataset_from_directory(val_path,label_mode='categorical', shuffle=False,image_size=(256, 256))

In [5]:
from tensorflow.keras.preprocessing import image
k=0
for i in os.listdir(train_path+'Buffalo/'):
  path = train_path + 'Buffalo/' + i
  img = image.load_img(path, target_size=(480, 480))
  plt.imshow(img)
  plt.show()
  k+=1
  if k==2:
    break

In [6]:
! pip install tf-nightly

In [7]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
from tensorflow.keras.applications.efficientnet_v2 import decode_predictions

In [8]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

base_model = EfficientNetV2L(weights='imagenet')
pred = base_model.predict(x)
print('Predicted:', decode_predictions(pred))

In [9]:
bmodel = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
bmodel.trainable = False

from tensorflow.keras import layers

x = bmodel.output
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model = Model(inputs=bmodel.input, outputs=outputs)

In [10]:
checkpoints = tf.keras.callbacks.ModelCheckpoint('EfficientNetV2L.h5',save_best_only=True, save_weights_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

m_fit = model.fit(train_df, epochs=20, validation_data=val_df, callbacks=[checkpoints])

In [11]:
result = m_fit.history

In [12]:
n_epochs = len(result['loss'])

plt.figure(figsize=[12,4])
plt.grid(True)
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), result['loss'], label='Training')
plt.plot(range(1, n_epochs+1), result['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.grid(True)
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), result['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), result['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [13]:
model.load_weights('EfficientNetV2L.h5')

In [14]:
pred = model.evaluate(test_df)

In [15]:
pred = model.predict(test_df)
pred

In [16]:
# save the index of the highest probability
pred = pred.argmax(axis=1)
print(pred)

In [17]:
# get the actual values
test_images = list(test_df.unbatch().as_numpy_iterator())
y_true = np.array([i[1] for i in test_images])
y_true = y_true.argmax(axis=1)
print(y_true)

In [18]:
# calculate f1_score
from sklearn.metrics import f1_score
f1_score(y_true,pred,average='macro')

In [19]:
# get the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true,pred)

In [20]:
%cd /kaggle/working
!ls

In [21]:
from IPython.display import FileLink
FileLink(r'EfficientNetV2L.h5')